In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [7]:
import os
from utils.general.read_files import read_from_json
from utils.general.save_files import save_to_json
from os.path import join
import shutil

In [8]:
def create_triplet_segmentation_per_image(ann_path_cholecinstanceseg,
                                          img_path_cholecinstanceseg,
                                          vid_ann_path_cholect50,
                                          ann_id_cholecT50,
                                          output_directory_vid_path):
    ann_vid_cholecT50 = read_from_json(vid_ann_path_cholect50)
    ann_cholecinstanceseg_img = read_from_json(ann_path_cholecinstanceseg)
    ann_name  = os.path.basename(ann_path_cholecinstanceseg)
    save_path =  join(output_directory_vid_path, 'ann_dir', ann_name)

    ann_cholecT50_img = ann_vid_cholecT50['annotations'][str(ann_id_cholecT50)]

    cholecT50_video_verb_dict = ann_vid_cholecT50['categories']['verb']
    cholecT50_video_target_dict = ann_vid_cholecT50['categories']['target']
    cholecT50_video_instrument_dict = ann_vid_cholecT50['categories']['instrument']

    ivt_info_in_cholecT50_img =  { cholecT50_video_instrument_dict[str(single_ivt_annotation[1])]:  {
                                                'verb': cholecT50_video_verb_dict[str(single_ivt_annotation[7])],
                                                'target': cholecT50_video_target_dict[str(single_ivt_annotation[8])]
        }   for single_ivt_annotation in ann_cholecT50_img if single_ivt_annotation[1] != -1}

    print(f'{ann_name} - ivt_info_in_cholecT50_img : {ivt_info_in_cholecT50_img} ')

    for ann_shape in ann_cholecinstanceseg_img['shapes']:
        label = ann_shape['label']
        ann_shape['verb'] =    ivt_info_in_cholecT50_img[label]['verb']
        ann_shape['target'] =    ivt_info_in_cholecT50_img[label]['target']
        
            
    save_to_json(data=ann_cholecinstanceseg_img,json_file_path=save_path)
    
    #save image. 
    shutil.copy(img_path_cholecinstanceseg, join(output_directory_vid_path, 'img_dir'))


In [9]:
def generate_triplet_segmentation_dataset(cholecinstanceseg_dataset, 
                                            cholect50_dataset, 
                                            accepted_frames_json_path, 
                                            triplet_segmentation_directory_path
                                            ):
    
    assert os.path.isdir(cholect50_dataset), 'the cholecT50 dataset path is not a directory' 
    assert os.path.isdir(cholecinstanceseg_dataset), 'the cholecinstanceseg dataset path is not a directory' 
    
    accepted_frames = read_from_json(accepted_frames_json_path)
    os.makedirs(triplet_segmentation_directory_path, exist_ok=True)
    
    for video_name_cholecinstanceseg, accepted_frame_names in accepted_frames.items():
        vid_name_cholecT50 =  video_name_cholecinstanceseg.split('_')[0]
        
        vid_ann_dir_cholecinstanceseg = join(cholecinstanceseg_dataset, video_name_cholecinstanceseg, 'ann_dir')
        vid_img_dir_cholecinstanceseg = join(cholecinstanceseg_dataset, video_name_cholecinstanceseg, 'img_dir')
        
        vid_ann_path_cholect50 = join(cholect50_dataset, 'labels',  f'{vid_name_cholecT50}.json') 
        
        triplet_segmentation_vid_path = join(triplet_segmentation_directory_path, video_name_cholecinstanceseg)
        os.makedirs(triplet_segmentation_vid_path , exist_ok=True)
        os.makedirs(join(triplet_segmentation_vid_path, 'img_dir') , exist_ok=True)
        os.makedirs(join(triplet_segmentation_vid_path, 'ann_dir') , exist_ok=True)

        for ann_name in accepted_frame_names:
            ann_id_cholecT50 = int(ann_name.split('.')[0].split('_')[-1])
            
            ann_path_cholecinstanceseg = join(vid_ann_dir_cholecinstanceseg, ann_name)
            img_path_cholecinstanceseg = join(vid_img_dir_cholecinstanceseg, ann_name.replace('json', 'png'))
            
            create_triplet_segmentation_per_image(ann_path_cholecinstanceseg,
                                                  img_path_cholecinstanceseg,
                                                  vid_ann_path_cholect50,
                                                  ann_id_cholecT50,
                                                  triplet_segmentation_vid_path)

I want to add information about the ivt into each shape.

In [10]:
triplet_segmentation_directory_path = '../data/triplet_segmentation_dataset'
accepted_frames_json_path = 'outputs/accepted_frames_per_dataset.json'
cholecinstanceseg_dataset = '../data/cholecinstanceseg'
cholect50_dataset = '../data/CholecT50/'

In [11]:
generate_triplet_segmentation_dataset(cholecinstanceseg_dataset, 
                                            cholect50_dataset, 
                                            accepted_frames_json_path, 
                                            triplet_segmentation_directory_path
                                            )

t50_VID01_000000.json - ivt_info_in_cholecT50_img : {'grasper': {'verb': 'grasp', 'target': 'gallbladder'}} 
t50_VID01_000001.json - ivt_info_in_cholecT50_img : {'grasper': {'verb': 'grasp', 'target': 'gallbladder'}} 
t50_VID01_000002.json - ivt_info_in_cholecT50_img : {'grasper': {'verb': 'grasp', 'target': 'gallbladder'}} 
t50_VID01_000003.json - ivt_info_in_cholecT50_img : {'grasper': {'verb': 'grasp', 'target': 'gallbladder'}} 
t50_VID01_000010.json - ivt_info_in_cholecT50_img : {'grasper': {'verb': 'grasp', 'target': 'gallbladder'}} 
t50_VID01_000011.json - ivt_info_in_cholecT50_img : {'grasper': {'verb': 'grasp', 'target': 'gallbladder'}} 
t50_VID01_000012.json - ivt_info_in_cholecT50_img : {'grasper': {'verb': 'grasp', 'target': 'gallbladder'}} 
t50_VID01_000013.json - ivt_info_in_cholecT50_img : {'grasper': {'verb': 'grasp', 'target': 'gallbladder'}} 
t50_VID01_000014.json - ivt_info_in_cholecT50_img : {'grasper': {'verb': 'grasp', 'target': 'gallbladder'}} 
t50_VID01_000015.js

## Add images